In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [62]:
import pandas as pd

sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
sales_data = sales.iloc[:, 6:].T  # Shape: (days, items) → (1913, 30490)
sales_data.columns = sales['id'].values

# === Load calendar.csv ===
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")

# === Select calendar columns used in training ===
calendar_cols = [
    'wday', 'month', 'year',
    'event_name_1', 'event_name_2', 'event_type_1',
    'snap_CA', 'snap_TX', 'snap_WI'
]

# Prepare full calendar_feats first (used for training)
calendar_train = calendar.iloc[:1913][calendar_cols].copy()
calendar_train = calendar_train.fillna("none")
calendar_feats = pd.get_dummies(calendar_train, columns=['event_name_1', 'event_name_2', 'event_type_1'])


# ✅ Save calendar feature columns for future alignment (important for validation/eval)
with open("calendar_columns.pkl", "wb") as f:
    pickle.dump(calendar_feats.columns.tolist(), f)
    
combined = pd.concat([sales_data.reset_index(drop=True), calendar_feats], axis=1)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined)


import numpy as np
# === Define sequence length ===
input_window = 28  # number of past days used as input
num_items = 30490  # number of items (sales columns only)
total_days = scaled_data.shape[0]  # total time steps available

# === Generate training sequences ===
X_train, y_train = [], []

for i in range(input_window, total_days):
    input_seq = scaled_data[i - input_window:i]  # shape: (28, features)
    target = scaled_data[i][:num_items]          # shape: (30490,), only sales part

    X_train.append(input_seq)
    y_train.append(target)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

print(f"✅ X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

✅ X_train shape: (1885, 28, 30537), y_train shape: (1885, 30490)


In [63]:
# === Extract next 28 days: day 1914 to 1941 (for validation) or 1942–1969 (for evaluation) ===
calendar_future = calendar.iloc[1913:1941][calendar_cols].copy()

# === Fill missing values and one-hot encode event-related categorical features ===
calendar_future = calendar_future.fillna("none")
calendar_future = pd.get_dummies(calendar_future, columns=['event_name_1', 'event_name_2', 'event_type_1'])

# === Ensure column alignment with training ===
calendar_future = calendar_future.reindex(columns=calendar_feats.columns, fill_value=0)

# === Convert to float32 NumPy array ===
calendar_next_28 = calendar_future.values.astype(np.float32)  # shape: (28, calendar_features)

In [64]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# === Define Model Architecture ===
def build_global_lstm_model(input_timesteps, num_features, num_items, dropout_rate=0.2):
    model = Sequential()

    # LSTM layers
    model.add(LSTM(units=64, return_sequences=True, input_shape=(input_timesteps, num_features)))
    model.add(Dropout(dropout_rate))

    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(dropout_rate))

    model.add(LSTM(units=128))  # Last layer: return_sequences=False
    model.add(Dropout(dropout_rate))

    # Output layer (no activation)
    model.add(Dense(units=num_items))

    # Optimizer
    optimizer = Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer, loss='huber')

    # Learning rate scheduler
    lr_callback = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    return model, lr_callback


In [65]:
# === Build and Train Model ===
model, lr_callback = build_global_lstm_model(
    input_timesteps=28,
    num_features=X_train.shape[2],
    num_items=30490
)

model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    callbacks=[lr_callback],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - loss: 0.0087 - val_loss: 0.0101 - learning_rate: 0.0010
Epoch 2/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0068 - val_loss: 0.0095 - learning_rate: 0.0010
Epoch 3/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - loss: 0.0065 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 4/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.0063 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 5/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0062 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 6/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0061 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 7/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.0061 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 8/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0060
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.0060 - val_loss: 0.0094 - learning_rate: 0.

In [67]:
# === Load calendar columns for alignment ===
with open("calendar_columns.pkl", "rb") as f:
    calendar_columns = pickle.load(f)

calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calendar_cols = ['wday', 'month', 'year', 'event_name_1', 'event_name_2', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI']

# === Prepare input and calendar features for validation ===
calendar_future_val = calendar.iloc[1913:1941][calendar_cols].copy()
calendar_future_val = calendar_future_val.fillna("none")
calendar_future_val = pd.get_dummies(calendar_future_val, columns=['event_name_1', 'event_name_2', 'event_type_1'])
calendar_future_val = calendar_future_val.reindex(columns=calendar_columns, fill_value=0)
calendar_next_28_val = calendar_future_val.values.astype(np.float32)

last_28_days_input_val = scaled_data[-28:, :].astype(np.float32)

# === Forecasting Function ===
def forecast_28_days(model, input_sequence, calendar_future, num_items):
    window = input_sequence.shape[0]
    input_data = input_sequence.copy()
    predictions = []

    for day in range(28):
        x_input = input_data[-window:].reshape(1, window, -1)
        y_pred = model.predict(x_input, verbose=0)

        # Inspect prediction values
        print("Prediction summary:", y_pred.min(), y_pred.max(), y_pred.mean())

        # Reconstruct full row for inverse transform (pad calendar zeros for inverse)
        full_pred_row = np.concatenate([y_pred[0], np.zeros(calendar_future.shape[1])]).reshape(1, -1)
        y_pred_original = scaler.inverse_transform(full_pred_row)[:, :num_items]

        # Add prediction (scaled again) + real calendar to next input
        y_pred_scaled = scaler.transform(full_pred_row)[:, :num_items]
        next_input_row = np.concatenate([y_pred_scaled[0], calendar_future[day]])
        input_data = np.vstack([input_data, next_input_row])

        predictions.append(y_pred_original)

    return np.vstack(predictions)

# === Forecast Validation Period ===
forecast_val = forecast_28_days(
    model=model,
    input_sequence=last_28_days_input_val,
    calendar_future=calendar_next_28_val,
    num_items=30490
)
forecast_val = np.clip(forecast_val, 0, None).T

# === Build submission DataFrame for validation ===
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
ids_val = sales['id'].values
submission_val_df = pd.DataFrame(forecast_val, columns=[f"F{i}" for i in range(1, 29)])
submission_val_df.insert(0, "id", ids_val)
submission_val_df.to_csv("submission_val.csv", index=False)
print("✅ submission_val.csv saved successfully!")

# === Prepare calendar for evaluation period ===
calendar_future_eval = calendar.iloc[1941:1969][calendar_cols].copy()
calendar_future_eval = calendar_future_eval.fillna("none")
calendar_future_eval = pd.get_dummies(calendar_future_eval, columns=['event_name_1', 'event_name_2', 'event_type_1'])
calendar_future_eval = calendar_future_eval.reindex(columns=calendar_columns, fill_value=0)
calendar_next_28_eval = calendar_future_eval.values.astype(np.float32)

last_28_days_input_eval = scaled_data[-28:, :].astype(np.float32)

# === Forecast Evaluation Period ===
forecast_eval = forecast_28_days(model, last_28_days_input_eval, calendar_next_28_eval, num_items=30490)
forecast_eval = np.clip(forecast_eval, 0, None).T

# === Build submission DataFrame for evaluation ===
ids_eval = [i.replace("validation", "evaluation") for i in sales["id"].values]
submission_eval_df = pd.DataFrame(forecast_eval, columns=[f"F{i}" for i in range(1, 29)])
submission_eval_df.insert(0, "id", ids_eval)
submission_eval_df.to_csv("submission_eval.csv", index=False)
print("✅ submission_eval.csv saved successfully!")

Prediction summary: -0.029571217 0.5268835 0.08579148
Prediction summary: -0.029586967 0.51839876 0.08463582
Prediction summary: -0.029538155 0.5313871 0.086427465
Prediction summary: -0.028945543 0.5371337 0.08721996


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.028069634 0.54042274 0.08768477
Prediction summary: -0.027162064 0.5418157 0.08791434
Prediction summary: -0.025391791 0.5419805 0.087837316
Prediction summary: -0.02267912 0.54171294 0.0875806


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.021080088 0.5378747 0.08683897
Prediction summary: -0.020914115 0.52920604 0.08555251
Prediction summary: -0.020282678 0.5212192 0.084387906
Prediction summary: -0.019309524 0.5157156 0.0835061


...

Prediction summary: -0.031733606 0.47833642 0.07706851
Prediction summary: -0.032998234 0.47799408 0.07704148
Prediction summary: -0.03474366 0.47751305 0.077047326


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.038756013 0.47896683 0.077215284
Prediction summary: -0.05239045 0.48423424 0.07736974


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


✅ submission_eval.csv saved successfully!


In [68]:
import pandas as pd

# === Load validation and evaluation prediction files ===
submission_val = pd.read_csv("submission_val.csv")
submission_eval = pd.read_csv("submission_eval.csv")

# === Combine both parts ===
submission_all = pd.concat([submission_val, submission_eval], axis=0).reset_index(drop=True)

# === Load sample_submission to match required order ===
sample = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

# === Align predictions to sample_submission id order ===
final_submission = sample[['id']].merge(submission_all, on='id', how='left')

# === Validate submission ===
num_duplicates = final_submission['id'].duplicated().sum()
num_missing = final_submission.isnull().sum().sum()

assert num_duplicates == 0, f"❌ Found {num_duplicates} duplicate ID(s) in submission"
assert num_missing == 0, f"❌ Found {num_missing} missing prediction(s) in submission"

# === Save final file ===
final_submission.to_csv("final_submission.csv", index=False)
print(f"✅ Final submission saved: {len(final_submission)} rows → final_submission.csv")

✅ Final submission saved: 60980 rows → final_submission.csv
